# 5가지 지표 점수 

ParkScore에 사용되는 지표 5가지의 점수 시각화 진행한다.
지표는 다음과 같다.
1. 접근
2. 면적
3. 투자
4. 공평
5. 시설   
  
85개의 도시의 점수를 명확하게 파악하기 위해서 "산점도"를 이용하여 나타낸다.

## Import

- 최종 ParkScore 점수 사용

In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

In [43]:
try:
    ps = pd.read_csv('./parkscore.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        ps = pd.read_csv('./parkscore.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [44]:
ps

,도,도시명,접근 점수,면적 점수,투자 점수,공평 점수,시설 점수,총 점수,순위
0,광역특별시,대구광역시,67,56,94,43,100,72,12
1,광역특별시,광주광역시,71,61,100,71,100,81,4
2,광역특별시,대전광역시,76,69,58,66,54,65,20
3,광역특별시,부산광역시,52,75,54,49,80,62,28
4,광역특별시,서울특별시,77,70,100,36,100,77,9
...,...,...,...,...,...,...,...,...,...
80,충청남도,아산시,45,39,100,36,100,64,22
81,충청남도,천안시,66,52,100,90,7,63,23
82,충청북도,제천시,63,10,100,26,67,53,49
83,충청북도,청주시,72,73,50,90,25,62,26


In [45]:
# 그래프 표현을 위한 열 이름 변경
ps.rename(columns={
    '접근 점수': 'Access',
    '면적 점수': 'Acreage',
    '투자 점수': 'Investment',
    '공평 점수': 'Equity',
    '시설 점수': 'Amenities',
    '총 점수' : 'Final_Score'
}, inplace=True)

In [46]:
ps

,도,도시명,Access,Acreage,Investment,Equity,Amenities,Final_Score,순위
0,광역특별시,대구광역시,67,56,94,43,100,72,12
1,광역특별시,광주광역시,71,61,100,71,100,81,4
2,광역특별시,대전광역시,76,69,58,66,54,65,20
3,광역특별시,부산광역시,52,75,54,49,80,62,28
4,광역특별시,서울특별시,77,70,100,36,100,77,9
...,...,...,...,...,...,...,...,...,...
80,충청남도,아산시,45,39,100,36,100,64,22
81,충청남도,천안시,66,52,100,90,7,63,23
82,충청북도,제천시,63,10,100,26,67,53,49
83,충청북도,청주시,72,73,50,90,25,62,26


## EDA

- 각 지표 별 점수 "산점도"를 이용하여 시각화하기 위한 함수 설정

In [47]:
def create_scatter_plot(df, metric, rgba_color):
    fig = px.scatter(df, x=metric, text='도시명', 
                     title=metric.upper(), labels={metric: 'SCORE'},
                     range_x=[-5, 105])

    # 텍스트 크기,위치 조정
    fig.update_traces(marker=dict(size=14, color=rgba_color), textfont_size=15,
                      hovertemplate='<b>%{text}</b><br>' + metric + ': %{x}<extra></extra>')

    # 배경색과 x축 간격 설정
    fig.update_layout(
        plot_bgcolor='rgba(242, 242, 242, 0.5)', 
        xaxis=dict(tickmode='linear', tick0=0, dtick=10),
        yaxis=dict(visible=False),
        title=dict(font=dict(size=24))  
    )

    # 겹침 방지
    fig.update_traces(textposition='top center')

    # html 형식 저장
    file_name = f"./five_metric/{metric}.html"
    fig.write_html(file_name)

    # 그래프 확인
    fig.show()

### 접근(Access)

In [56]:
# 접근 
create_scatter_plot(ps, 'Access', 'rgba(0, 114, 186, 1)')

### 면적(Acreage)

In [57]:
# 면적 
create_scatter_plot(ps, 'Acreage', 'rgba(190, 214, 1, 1)')

### 투자(Investment)

In [58]:
# 투자
create_scatter_plot(ps, 'Investment', 'rgba(101, 172, 0, 1)')

### 공평(Equity)

In [59]:
# 공평
create_scatter_plot(ps, 'Equity', 'rgba(142, 200, 59, 1)')

### 시설(Amenities)

In [60]:
# 시설
create_scatter_plot(ps, 'Amenities', 'rgba(254, 200, 56, 1)')

## 문제 해결

- 투자와 시설 지표는 100점이 다수 존재하여 수정이 필요하다.

In [86]:
def modified_plot(df, metric, rgba_color):
    
    # 무작위적인 분산 추가(100점 부분 해결)
    jitter = np.random.uniform(-0.5, 1, size=len(df))
    df['jittered_metric'] = df[metric] + jitter

    # 텍스트 크기를 조정하기 위한 조건부 컬럼 추가
    df['text_size'] = np.where(df[metric] == 100, 12, 15)  # 100점인 경우 텍스트 크기를 12로, 그렇지 않은 경우 15로 설정
    
    fig = px.scatter(df, x='jittered_metric', text='도시명', 
                     title=metric.upper(), labels={'jittered_metric': 'SCORE'},
                     range_x=[-5, 105])

    # 텍스트 크기, 위치 조정
    fig.update_traces(marker=dict(size=14, color=rgba_color), textfont_size=df['text_size'],
                      hovertemplate='<b>%{text}</b><br>' + metric + ': %{customdata}<extra></extra>',
                      customdata=df[metric])

    # 배경색과 x축 간격 설정
    fig.update_layout(
        plot_bgcolor='rgba(242, 242, 242, 0.5)', 
        xaxis=dict(tickmode='linear', tick0=0, dtick=10),
        yaxis=dict(visible=False),
        title=dict(font=dict(size=24))  
    )

    # 겹침 방지
    fig.update_traces(textposition='top center')

    # html 형식 저장
    file_name = f"./five_metric/{metric}.html"
    fig.write_html(file_name)

    # 그래프 확인
    fig.show()

In [87]:
# 투자
modified_plot(ps, 'Investment', 'rgba(101, 172, 0, 1)')

In [88]:
# 시설
modified_plot(ps, 'Amenities', 'rgba(254, 200, 56, 1)')

## 도시의 최종 점수(5가지 지표 합산값)

In [15]:
# 최종 점수
create_scatter_plot(ps, 'Final_Score', 'rgba(180, 85, 162, 0.7)')